In [1]:
import numpy as np
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import pandas as pd

In [272]:
def rotate_points_3d(points3d, rotX, rotY, rotZ):
    r = Rotation.from_euler('XYZ', [rotX, rotY, rotZ], degrees=True)
    return r.apply(points3d)

def get_point_on_ellipse(a, b, theta):
    a, b = max(a, b), min(a, b)
    e = (1 - b**2 / a ** 2) ** 0.5
    c = a * e
    r = a * (1 - e * e) / (1 - e * np.cos(theta))
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y

# G=6.67408 * 10**(-11)
def get_required_velocity(mass, distance_from_center_of_mass, a, b, body_mass, G=1):
    if distance_from_center_of_mass == 0 :
        print('get_required_velocity: distance_from_center_of_mass =', distance_from_center_of_mass)
        return 0
    ret = np.sqrt(G*mass*(2/distance_from_center_of_mass - 1 / (max(a, b))))
    return ret

In [333]:
class ConstPlane:
    def __init__(self, position=np.array([0,0,0]), velocity=np.array([0,0,0])):
        self.position = np.array(position)
        self.velocity = np.array(velocity)
        
    def update_velocity(*args, **kwargs):
        pass
    
    def as_dict(self):
        return {
            'position': self.position,
            'velocity': self.velocity
        }
    
    def __str__(self):
        return str(self.as_dict())
    
    __repr__ = __str__

class ElipticPlane:
    def __init__(self, a, b, theta=0, rotations=[0,0,0], move_clockwise=True, dtheta=0.001):
        self._a = a
        self._b = b
        self._theta = theta
        self._dtheta = dtheta
        self._rotations = rotations
        self._move_clockwise = move_clockwise
        
        pos = get_point_on_ellipse(self._a, self._b, self._theta)
        
        self.position = rotate_points_3d(np.array([pos[0], pos[1], 0]), 
                                *self._rotations)
        
        pos1 = np.array(get_point_on_ellipse(a, b, theta + dtheta))
        pos2 = np.array(get_point_on_ellipse(a, b, theta - dtheta))
        if move_clockwise:
            vel = pos2 - pos1
        else:
            vel = pos1 - pos2
            
        vel = rotate_points_3d(np.array([vel[0], vel[1], 0]), *self._rotations)
        self._velocity_normed = vel / np.sqrt(np.sum(vel*vel))
        self.velocity = self._velocity_normed

    
    def update_velocity(self, center_of_mass, central_mass, body_mass):
        diff_pos = center_of_mass - self.position
#         distance_from_center = np.sqrt(np.sum(diff_pos*diff_pos))
        pos = np.array(get_point_on_ellipse(self._a, self._b, self._theta))
#         print(pos)
        distance_from_center = np.sqrt(np.sum(pos*pos))
        self.velocity = self._velocity_normed * get_required_velocity(central_mass, distance_from_center, self._a, self._b, body_mass)
        
    def copy(self):
        return ElipticPlane(self._a, self._b, self._theta, 
                            self._rotations, self._align_focus, self._move_clockwise)
    
    def as_dict(self):
        return {
            'a': self._a,
            'b': self._b,
            'theta': self._theta,
            'pos': self.position,
            'rot': self._rotations
        }
    
    def __str__(self):
        return str(self.as_dict())
    
    __repr__ = __str__
    
    
class SpaceObject:
    def __init__(self, mass, plane, color=[255, 255, 255]):
        self.plane = plane
        self.mass = mass
        self.position_offset = np.array([0,0,0])
        self.velocity_offset = np.array([0,0,0])
        self.color = color
        self._central_mass = 0
    
    @property
    def center_of_mass(self):
        return self.plane.position + self.position_offset
    
    @property
    def velocity(self):
        return self.plane.velocity + self.velocity_offset
    
    def update_speed(self, center_of_mass, central_mass, offset=[0,0,0]):
        self._central_mass = central_mass
        self.plane.update_velocity(center_of_mass, self._central_mass, self.mass)
        self.velocity_offset = np.array(offset)
    
    def update_position(self, offset=[0,0,0]):
        self.position_offset = offset
    
    def agg(self):
        pos = self.center_of_mass
        vel = self.velocity
        return [{
            'px': pos[0],
            'py': pos[1],
            'pz': pos[2],
            'vx': vel[0],
            'vy': vel[1],
            'vz': vel[2],
            'm': self.mass,
            'r': self.color[0],
            'g': self.color[1],
            'b': self.color[2]
        }]
    
    def copy(self):
        return SpaceObject(self.mass, self.plane.copy())
    
    def as_dict(self):
        return {
            'mass': self.mass,
            'color': self.color,
            'plane': self.plane,
            'position_offset': self.position_offset,
            'velocity_offset': self.velocity_offset
        }
    
    def __str__(self):
        return str(self.as_dict())
    
    __repr__ = __str__

class SpaceSystem:
    def __init__(self, objects, plane):
        self.objects = objects
        self.plane = plane
        self.velocity_offset = np.array([0,0,0])
        self.position_offset = np.array([0,0,0])
        self.mass = np.sum([el.mass for el in self.objects])
    
    @property
    def center_of_mass(self):
        global ii
        mass = self.mass
        return self.position_offset + np.sum([el.center_of_mass * el.mass for el in self.objects], axis=0) / mass
    
    def update_speed(self, center_of_mass, central_mass, offset=np.array([0,0,0])):
        self.velocity_offset = offset
        
        center = self.center_of_mass
        mass = self.mass
        
        self.plane.update_velocity(center_of_mass, central_mass, self.mass)
        velocity_and_offset = self.plane.velocity + offset
        
        for obj in self.objects:
            obj.update_speed(center, mass, velocity_and_offset)
    
    def update_position(self, offset=[0,0,0]):
        self.position_offset = offset
        for obj in self.objects:
            obj.update_position(self.plane.position)
        
    def init_update(self):
        center = self.center_of_mass
        mass = self.mass
        for obj in self.objects:
            obj.update_position()
            obj.update_speed(center, mass)
    
    def agg(self):
        ret_val = []
        for obj in self.objects:
            ret_val.extend(obj.agg())
        return ret_val
    
    def copy(self):
        return SpaceSystem([obj.copy() for obj in self.objects], self.plane.copy())
    
    def as_dict(self):
        return {
            "objects": [el.as_dict() for el in self.objects],
            'plane': self.plane
        }
    
    def __str__(self):
        return str(self.as_dict())
    
    __repr__ = __str__

In [334]:
const_plane1 = ConstPlane()
circle_plane1 = ElipticPlane(100, 100, -np.pi/2)
obj1 = SpaceObject(1000000, const_plane1, color=[235, 120, 0])
circle_plane2 = ElipticPlane(1000, 200, np.pi*0)
obj2 = SpaceObject(1, circle_plane2, color=[255, 0, 0])

system_objs1 = SpaceSystem([obj1, obj2], None)
system_objs1.init_update()

In [335]:
data_aggregated = system_objs1.agg()

In [336]:
df = pd.DataFrame(data_aggregated)
df

,px,py,pz,vx,vy,vz,m,r,g,b
0,0.000000,0.0,0.0,0.0,0.000000,0.0,1000000,235,120,0
1,1979.795897,0.0,0.0,0.0,-3.194551,0.0,1,255,0,0


In [337]:
df.to_csv("../TestData_v2/2 points system.txt", index=False, sep='\t')

## Массовая генерация

In [393]:
def generate_haotic_asteroid_space(objects, a, b, a_delta, b_delta, plane_rotations, plane_delta_rotations, move_clockwise=True):
    np.random.seed(42)
    placed_objects = []
    plane_rotations = np.array(plane_rotations)
    plane_delta_rotations = np.array(plane_delta_rotations)
    
    cur_rot = plane_rotations + ((np.random.random((len(objects), 3)) * 2 - 1) * plane_delta_rotations)
    cur_theta = np.random.random(len(objects)) * 2 * np.pi
    a_s = a + (2 * np.random.random(len(objects)) - 1) * a_delta
    b_s = b + (2 * np.random.random(len(objects)) - 1) * b_delta
    print(a_s.min(), a_s.max(), a_s.mean())
    print(b_s.min(), b_s.max(), b_s.mean())
    
    for ind, obj in enumerate(objects):
        s = SpaceSystem([obj], ElipticPlane(a_s[ind], b_s[ind], 
                                            theta=cur_theta[ind], 
                                            rotations=cur_rot[ind],  
                                            move_clockwise=move_clockwise))
        placed_objects.append(s)
    return placed_objects
#     return SpaceSystem(placed_objects, ConstPlane())

## Генерация данных симуляции/

In [339]:
sun_plane = ConstPlane()
sun_obj = SpaceObject(1000000, sun_plane, color=[255, 255, 0])

masses = [1,   1.5,  3,    2.7,   50,   30,   10, 15]
a_s =    [500, 1200, 2300, 2800, 4400, 7000, 12000, 17000]
b_s =    [400, 1300, 2500, 3000, 4200, 7500, 11900, 16700]
thetas = np.linspace(0, np.pi*2, 8)
directions = [True, True, True, True, True, True, True, False]
colors = [
    [255, 123, 0],
    [130, 114, 91],
    [144, 222, 0],
    [219, 150, 53],
    [176, 122, 14],
    [85, 224, 194],
    [120, 92, 66],
    [102, 207, 255]
]

planets_objs = [
    SpaceObject(mass, 
                       ElipticPlane(a, b, theta, move_clockwise=direction), 
                       color=color)
    for (mass, a, b, theta, color, direction) in zip(masses, a_s, b_s, thetas, colors, directions)
]


In [340]:
system_objs1 = SpaceSystem([sun_obj, *planets_objs], None)
system_objs1.init_update()
data_aggregated = system_objs1.agg()
df = pd.DataFrame(data_aggregated)
display(df)
df.to_csv("../TestData_v2/7 planet system.txt", index=False, sep='\t')

,px,py,pz,vx,vy,vz,m,r,g,b
0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,1000000.0,255,255,0
1,800.000000,0.000000e+00,0.0,0.000000,-22.361945,0.0,1.0,255,123,0
2,908.495504,1.139217e+03,0.0,23.492440,-7.177695,0.0,1.5,130,114,91
3,-433.084942,1.897469e+03,0.0,21.195287,13.358129,0.0,3.0,144,222,0
4,-1779.076078,8.567579e+02,0.0,8.487907,24.648516,0.0,2.7,219,150,53
5,-2847.402808,-1.371237e+03,0.0,-6.852910,18.937961,0.0,50.0,176,122,14
6,-1346.254576,-5.898327e+03,0.0,-12.062288,7.194997,0.0,30.0,85,224,194
7,8000.319459,-1.003208e+04,0.0,-7.197495,-4.553808,0.0,10.0,120,92,66
8,20179.622619,-4.942582e-12,0.0,0.000000,6.347512,0.0,15.0,102,207,255


## Генерация системы астероидных поясов

In [341]:
total_asteroids = 1000
asteroids_colors = np.array([
    [145, 125, 99],
    [109, 114, 117],
    [199, 179, 255]
])
asteroid_min_max_masses = [0.001, 0.1]
masses = asteroid_min_max_masses[0] + np.random.random(total_asteroids) * asteroid_min_max_masses[1] - asteroid_min_max_masses[0]
asteroids = [SpaceObject(masses[i], ConstPlane(), asteroids_colors[np.random.randint(0, len(asteroids_colors))]) 
             for i in range(total_asteroids)]

In [456]:
# %%prun
# asteroids_placed = generate_haotic_asteroid_space(asteroids, 5000, 5400, 10, 10, [5, 10, 0], [1, 1, 1])

asteroids_placed = generate_haotic_asteroid_space(asteroids[:total_asteroids//2], 5000, 5300, 50, 50, [5, 15, 0], [5, 5, 5])
# asteroids_placed = generate_haotic_asteroid_space(asteroids[:total_asteroids//2], 5000, 5300, 50, 50, [5, 15+90, 0], [5, 5, 5])
asteroids_placed.extend(generate_haotic_asteroid_space(asteroids[total_asteroids//2:], 5000, 5300, 50, 50, [5, 15+90, 0], [5, 5, 5]))

4950.06533907613 5049.955770325044 4999.191739821643
5250.003071884538 5349.7749389010205 5298.681520597496
4950.06533907613 5049.955770325044 4999.191739821643
5250.003071884538 5349.7749389010205 5298.681520597496


In [457]:
# %%prun
entireSystem = SpaceSystem([sun_obj,  *asteroids_placed], None)
entireSystem.init_update()

In [458]:
data_aggregated = entireSystem.agg()
df = pd.DataFrame(data_aggregated)
display(df)
df.to_csv("../TestData_v2/asteroidal system.txt", index=False, sep='\t')

,px,py,pz,vx,vy,vz,m,r,g,b
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+07,255,255,0
1,5016.785869,3990.098774,-1519.841711,25.800811,-21.521878,-10.568613,2.076518e-06,199,179,255
2,5242.087179,-3750.793929,-1471.466600,-26.371517,-22.909256,3.021343,2.607088e-04,84,65,138
3,-1796.600523,-3560.893766,570.710999,-40.146967,38.322771,13.948349,5.182797e-04,109,114,117
4,3033.384921,-4599.251952,-1121.604507,-40.396089,-11.020640,5.959621,7.222161e-04,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1496,-1221.348030,-4364.853951,-4035.318553,11.874683,-16.399144,33.120166,3.689876e-04,109,114,117
1497,-627.629579,5015.486304,-2786.926983,-8.602472,-8.220595,-39.259270,8.546190e-04,199,179,255
1498,515.530271,3783.546779,1841.372882,-11.234807,36.416744,-39.409217,8.022030e-04,84,65,138
1499,200.040012,4142.469920,1367.122987,-12.592100,30.314556,-42.260294,3.487612e-04,84,65,138


## Генерация системы из астероидов и планет

In [463]:
sun_plane = ConstPlane()
sun_obj = SpaceObject(10_000_000, sun_plane, color=[255, 255, 0])

masses = [15,   250,  300,    40,   10_000,   3_000,   500, 560]
a_s =    [500, 1200, 2400, 2800, 5400, 8400, 12000, 17000]
b_s =    [450, 1300, 2500, 3000, 5450, 8500, 11900, 16700]
thetas = np.linspace(0, np.pi*2, 8)
directions = [True, True, True, True, True, False, True, False]
colors = [
    [255, 123, 0],
    [130, 114, 91],
    [144, 222, 0],
    [219, 150, 53],
    [176, 122, 14],
    [85, 224, 194],
    [120, 92, 66],
    [102, 207, 255]
]

mult_factor = 8

planets_objs = [
    SpaceObject(mass, 
                       ElipticPlane(mult_factor*a, mult_factor*b, theta, move_clockwise=direction), 
                       color=color)
    for (mass, a, b, theta, color, direction) in zip(masses, a_s, b_s, thetas, colors, directions)
]


total_asteroids = 1500
asteroids_colors = np.array([
#     [145, 125, 99],
    [109, 114, 117],
    [199, 179, 255],
    [84, 65, 138],
    [0,0,0]
])
asteroid_min_max_masses = [0.0001, 0.001]

colors = asteroids_colors[np.random.randint(0, len(asteroids_colors) - 1, total_asteroids)]
masses = asteroid_min_max_masses[0] + np.random.random(total_asteroids) * asteroid_min_max_masses[1] - asteroid_min_max_masses[0]
mask_big_asteroids = np.where(np.random.random(len(masses))>0.98, 1, 0)
masses[mask_big_asteroids == 1] = len(asteroids_colors) - 1
masses *= mask_big_asteroids * 5999 + 1

asteroids = [SpaceObject(masses[i], ConstPlane(), asteroids_colors[np.random.randint(0, len(asteroids_colors))]) 
             for i in range(total_asteroids)]


asteroids_placed = generate_haotic_asteroid_space(asteroids[:int(0.6*total_asteroids)], mult_factor*4200, mult_factor*4200, mult_factor*90, mult_factor*90, [5, 7, 0], [10, 10, 10])
# asteroids_placed = generate_haotic_asteroid_space(asteroids[:total_asteroids//2], 5000, 5300, 50, 50, [5, 15+90, 0], [5, 5, 5])
asteroids_placed.extend(generate_haotic_asteroid_space(asteroids[int(0.6*total_asteroids):], mult_factor*7500, mult_factor*7500, mult_factor*80, mult_factor*80, [-5, -5, 180], [10, 10, 10], move_clockwise=False))

32880.04423513735 34319.36309268063 33578.9444809381
32880.32693503429 34318.26182757454 33599.19190039482
59360.01489248687 60637.21069527293 60006.19875729543
59362.54993308041 60638.58254340261 59994.04619212208


In [464]:
entireSystem = SpaceSystem([sun_obj, *planets_objs, *asteroids_placed], None)
entireSystem.init_update()
data_aggregated = entireSystem.agg()
df = pd.DataFrame(data_aggregated)
display(df.head(20))
df.to_csv("../TestData_v2/planet and asteroidal system.txt", index=False, sep='\t')

,px,py,pz,vx,vy,vz,m,r,g,b
0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000e+07,255,255,0
1,5743.559577,0.000000e+00,0.000000,0.000000,-31.976478,0.000000,1.500000e+01,255,123,0
2,7267.964035,9.113739e+03,0.000000,26.797718,-8.187563,0.000000,2.500000e+02,130,114,91
3,-3860.946271,1.691591e+04,0.000000,23.169974,11.942825,0.000000,3.000000e+02,144,222,0
4,-14232.608625,6.854063e+03,0.000000,9.682116,28.116448,0.000000,4.000000e+01,219,150,53
5,-34378.736797,-1.655593e+04,0.000000,-6.766626,16.158709,0.000000,1.000000e+04,176,122,14
6,-14291.120161,-6.261349e+04,0.000000,12.206663,-4.701002,0.000000,3.000000e+03,85,224,194
7,64002.555670,-8.025667e+04,0.000000,-8.210149,-5.194508,0.000000,5.000000e+02,120,92,66
8,161436.980953,-3.954066e-11,0.000000,0.000000,7.240578,0.000000,5.600000e+02,102,207,255
9,-25490.685996,-6.097881e+03,7057.875862,-3.944352,20.947938,2.044397,9.262082e-04,0,0,0
